# Processing the labels of the raw IMDB data

In [1]:
import os
train_dir = 'E:/aclImdb/aclImdb/train'
labels = []
texts = []

In [2]:
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname),encoding ='utf-8' )
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [3]:
len(texts)

25000

In [4]:
len(labels)

25000

# Tokenizing the text of the raw IMDB data

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [6]:
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000

In [7]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 88582 unique tokens.


In [8]:
data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [9]:
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

# Parsing the GloVe word-embeddings file

In [19]:
glove_dir = 'E:/glove.6B'

embeddings_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'),encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


# Preparing the GloVe word-embeddings matrix

In [20]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

# Model definition

In [21]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 32)                320032    
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


# Loading pretrained word embeddings into the Embedding layer

In [22]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# Training and evaluation

In [24]:
model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Epoch 1/100
7/7 [==============================] - 2s 168ms/step - loss: 1.1405 - acc: 0.8350 - val_loss: 0.8783 - val_acc: 0.5612
Epoch 2/100
7/7 [==============================] - 1s 133ms/step - loss: 0.0104 - acc: 1.0000 - val_loss: 0.8834 - val_acc: 0.5624
Epoch 3/100
7/7 [==============================] - 1s 138ms/step - loss: 0.0072 - acc: 1.0000 - val_loss: 0.8857 - val_acc: 0.5622
Epoch 4/100
7/7 [==============================] - 1s 138ms/step - loss: 0.0056 - acc: 1.0000 - val_loss: 0.9291 - val_acc: 0.5578
Epoch 5/100
7/7 [==============================] - 1s 140ms/step - loss: 0.0043 - acc: 1.0000 - val_loss: 0.9488 - val_acc: 0.5546
Epoch 6/100
7/7 [==============================] - 1s 141ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.9019 - val_acc: 0.5624
Epoch 7/100
7/7 [==============================] - 1s 139ms/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.8959 - val_acc: 0.5621
Epoch 8/100
7/7 [==============================] - 1s 142ms/step - loss: 0.0020 - a

7/7 [==============================] - 1s 135ms/step - loss: 1.5256e-06 - acc: 1.0000 - val_loss: 1.5198 - val_acc: 0.5659
Epoch 62/100
7/7 [==============================] - 1s 142ms/step - loss: 1.4317e-06 - acc: 1.0000 - val_loss: 1.5040 - val_acc: 0.5674
Epoch 63/100
7/7 [==============================] - 1s 135ms/step - loss: 1.3561e-06 - acc: 1.0000 - val_loss: 1.4915 - val_acc: 0.5687
Epoch 64/100
7/7 [==============================] - 1s 142ms/step - loss: 1.2717e-06 - acc: 1.0000 - val_loss: 1.5495 - val_acc: 0.5642
Epoch 65/100
7/7 [==============================] - 1s 144ms/step - loss: 1.1796e-06 - acc: 1.0000 - val_loss: 1.5332 - val_acc: 0.5648
Epoch 66/100
7/7 [==============================] - 1s 138ms/step - loss: 1.0613e-06 - acc: 1.0000 - val_loss: 1.5544 - val_acc: 0.5641
Epoch 67/100
7/7 [==============================] - 1s 136ms/step - loss: 9.4558e-07 - acc: 1.0000 - val_loss: 1.5631 - val_acc: 0.5638
Epoch 68/100
7/7 [==============================] - 1s 146ms/

In [25]:
#model.save_weights('pre_trained_glove_model.h5')

# Plotting the results

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
               
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
               
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()